In [2]:
import requests, math, re, sys, time, json
from bs4 import  BeautifulSoup as bs

In [3]:
rs = requests.session()
res = rs.get('http://vacation.eztravel.com.tw/pkgfrn/results/TPE/48?fullStatus=NONE')
soup = bs(res.text, "lxml")

In [4]:
count  = soup.select('.v-line')[0].text.split(' ')
print count[1]
page = int(math.ceil(float(count[1]) / 18))
print page

245
14


In [7]:
urlFormat = 'http://vacation.eztravel.com.tw/pkgfrn/results/TPE/48/{}?&fullStatus=NONE'
with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt','w') as f:
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt
    for pageNum in range(1, page + 1):
        url = urlFormat.format(pageNum)
        res2 = rs.get(url)
        soup1 = bs(res2.text, "lxml")
        list_table = soup1.select('div#listTable')[0]
        for items in list_table.select('div.list-box.mainLinkBox'):
            f.write('http://vacation.eztravel.com.tw/pkgfrn/' + items.select('a')[0]['href'].split('/pkgfrn/')[1] + '\n')

In [8]:
dicCheck = {} #宣告一個dic，稍後會將檔名(Key)跟網址(Value)一一放入
num = 0
for bid_url in open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt','r'): #讀連結檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt
    num += 1
    splitStr = bid_url.split('/') #將網址切割，並且丟入splitStr這個List裡
    linkName = splitStr[5].strip() + "_" + splitStr[6].strip() #將List[index]對應到的元素，以字串型式組合成dataNmae
    #用商品編號當作檔名
    if linkName not in dicCheck:
        print 'No.' + str(num) + ', ' + linkName +  ' is new, and added in dicCheck.'
        dicCheck[linkName] = bid_url
    else:
        print 'No.' + str(num) + ', ' + linkName + ' is existed, and not added in dicCheck.'
        print linkName, dicCheck[linkName] #若有重複，則print出重複的Key跟Value

No.1, VDR0000007986_SPK04GE60216E_6701 is new, and added in dicCheck.
No.2, VDR0000001843_SPKGE16060305A is new, and added in dicCheck.
No.3, VDR0000001843_SPKGE16061405A is new, and added in dicCheck.
No.4, VDR0000007986_SPK05GE60616E_6830 is new, and added in dicCheck.
No.5, VDR0000001888_SPK05160530E is new, and added in dicCheck.
No.6, VDR0000001975_HKD04160613TA is new, and added in dicCheck.
No.7, VDR0000001279_DTS-A0602HAS is new, and added in dicCheck.
No.8, VDR0000001838_NNZ060304BR6E is new, and added in dicCheck.
No.9, VDR0000001846_16JHO22TZ is new, and added in dicCheck.
No.10, FRN0000014017_20160602 is new, and added in dicCheck.
No.11, VDR0000001279_DTS-A0601HHC is new, and added in dicCheck.
No.12, VDR0000001279_DTS-A0604HAH is new, and added in dicCheck.
No.13, VDR0000001943_HKD04IT6814A is new, and added in dicCheck.
No.14, VDR0000001888_SPK05160601G is new, and added in dicCheck.
No.15, VDR0000007986_SPK05GE60616E_6812 is new, and added in dicCheck.
No.16, VDR0000001

In [9]:
### 印出字典的key與value########
def PrintKeyValue(dic_in):
    for key, value in dic_in.iteritems():
        print key, ':', value 

In [10]:
itemDetail = {} #放置所有細項資料集合
eachTravel = [] #放置所有細項資料於旅行方案集合
totalTravel = {} #放置所有的旅行方案於總集合
totalCount = 0
errorCount = 0
with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Hokkaido.txt', 'w') as errorFile:
    #寫error檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Hokkaido.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Hokkaido.txt
    for bid_url in open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Hokkaido.txt', 'r'):
        #讀連結檔
        #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
        #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
        try: #使用try-except
            daysHotFormat = 'day{}_Hotel' #建立字串格式
            daysTourFormat = 'day{}_Tour' #同上
            bid_detail = requests.get(bid_url.strip()) #讀取檔案中的連結
            soup2 = bs(bid_detail.text, "lxml") #萃取資料
            row = soup2.select('div#pkgfrnVisitHistory')[0]
            price = re.search('\d+', row.select('.price')[0].text) #利用正規表示法抓取價格
            plane = re.search('.\W{2,4}', row.select('div#flight-block1 span')[3].text) #利用正規表示法抓取航空公司名稱
            daysTransfer = (row.select('div.product-info.css-td')[0].text.strip().split(u'天數')) #抓每筆旅行方案的旅行天數
            days = (re.search('\d{1}', daysTransfer[1])) #利用正規表示法抓出旅行天數
            itemDetail['Title'] = row.select('div.pro-title.css-td h1')[0].text #將個別資料丟入itemDetail字典中，以下相同
            itemDetail['Price'] = price.group(0)
            itemDetail['City'] = row.select('.tag')[0].text.strip()
            itemDetail['Time'] = row.select('div#flight-block1 span')[2].text.strip()
            itemDetail['PlaneCom'] = plane.group(0)

            if len(row.select('div.pro-id-right.css-td span')[0].text.strip()) > 0: #判斷若PID_1長度大於零，則將資料丟入
                itemDetail['PID_1'] = row.select('div.pro-id-right.css-td span')[0].text.strip()
            else: #判斷若PID_1長度小於零，則丟入Null
                itemDetail['PID_1'] = 'Null'
            if len(row.select('div.pro-id-right.css-td span')[1].text.strip()) > 0: #同上
                itemDetail['PID_2'] = row.select('div.pro-id-right.css-td span')[1].text.strip()
            else: #同上
                itemDetail['PID_2'] = 'Null'
            totalCount += 1 #計算有多少筆資料丟入JSON檔中

            print '----------------------------這是分隔線----------------------------'
            print 'days: ' + days.group(0) #印出當前旅行方案的旅行天數
            print 'Url: ' + bid_url.strip() #印出當前旅行方案的網址

            #因為EZtravel的網站資料較鬆散，且只有旅館跟行程有規律性，故特別獨立出來寫加入的方法
            num = 1 #因住宿飯店的索引值是[1]、[3]、[5]……的奇數，所以給起始值，每執行一次會增加2
            for times in range(0,int(days.group(0))):
                daysHotInput = daysHotFormat.format(times + 1) #這邊是天數，day{1}_Hotel、day{2}_Hotel……
                itemDetail[daysHotInput] = row.select('div.meals')[num].text.strip()
                #這邊是索引值(奇數)，row.select('div.meals')[1].text.strip()、row.select('div.meals')[3].text.strip()……
                num += 2
            for indexTour in range(0,int(days.group(0))):
                daysTourInput = daysTourFormat.format(indexTour + 1) #這邊是天數，day{1}_Tour、day{2}_Tour……
                itemDetail[daysTourInput] = row.select('div.day-box h4')[indexTour].text.strip()
                #這邊是索引值，row.select('div.day-box h4')[1].text.strip()、row.select('div.day-box h4')[2].text.strip()……
            PrintKeyValue(itemDetail)
            eachTravel.append(itemDetail)
            totalTravel['eachTravel'] = eachTravel

        except:
            errorCount += 1
            splitPID = bid_url.strip().split('/')
            strPID = splitPID[5].strip() + "_" + splitPID[6].strip()
            #當錯誤訊息出現時，印出是哪一筆資料出現錯誤
            print '----------------' + strPID + ' is error----------------'
                #將出現錯誤的資料網址，寫入txt檔中
            errorFile.write(bid_url)

with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Hokkaido.json', 'w') as dataFile:
    #寫最後全部資料檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Hokkaido.json
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Hokkaido.json
    json.dump(totalTravel, dataFile)

print '----------------------------All datas are done!!!----------------------------'
print 'Data Quantities: ' + str(totalCount)
print 'Error Quantities: ' + str(errorCount)

----------------------------這是分隔線----------------------------
days: 4
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000007986/SPK04GE60216E_6701
City : 札幌
PID_1 : VDR0000007986
day4_Hotel : 溫暖的家
day4_Tour : 札幌市區觀光【時計台、大通公園、舊道廳】－北海道神宮－千歲空港／桃園
PlaneCom : 復興航空 
PID_2 : SPK04GE60216E_6701
Price : 20800
Title : 【旅展破盤~北海道19900起含稅】美瑛夢幻青池、富良野花海、白色戀人公園4日(千千)
day1_Hotel : 十勝第一 或 大平原 或 幕別 GREEN 或夕張溫泉 shuparo飯店 或 同級
day3_Hotel : 札幌  花園皇宮  或 札幌APA  RESORT~大浴場.露天風呂 或 札幌CHISIN GRAND 或 同級飯店
day2_Tour : 日本的普羅旺斯．富田農場．彩虹花田－美瑛夢幻私房景點．白金清池－道央的廣告拍攝地．美瑛拼布．四季彩之丘．沿途欣賞拼布之路美景－旭川夜風華
day2_Hotel : 旭川 クレッセント或 ART 或 DORMY INN 或同級
Time : 07:30 - 12:10
day3_Tour : 浪漫天堂~小樽運河、北一硝子館、音樂盒館、百年蒸氣鐘、銀之鐘－白色戀人公園《漫步英國風城堡》－免稅店－札幌 ~晚上您可自由前往狸小路逛街購物
day1_Tour : 桃園／千歲空港－啤酒廠見學．試飲美味啤酒－十勝川溫泉．美人湯露天浴場體驗．日本唯一之植物性MORU溫泉對美化肌膚治療神經痛有特效
----------------------------這是分隔線----------------------------
days: 5
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001843/SPKGE16060305A
City : 札幌
PID_1 : VDR0000001843
day4_Hotel 